In [ ]:
import numpy as np
import os
import h5py
import sima
from sima import sequence
from sima.imaging import ImagingDataset
import matplotlib.pyplot as plt

import bidi_offset_correction
import sima_motion_bidi_correction as bidi_funcs

In [ ]:
fname = '22_1_19_gck_cgm_3_sacchari-001' # h5 file name (without the extension)
fdir = r'D:\bruker_data\Jennifer\22_1_19_gck_cgm_3_sacchari-001' # root folder of h5 to cut
data_format = 'sima'
manual_offset_value = 10 # in pixels (I found this to be pixels you measure by eye +1)

num_save_frames = 300 # use the following for saving and applying offset to all frames: data_n_meta['data'].shape[0] 

In [ ]:
# load an object referencing the motion-corrected data
data_n_meta = bidi_funcs.load_sima_or_h5_object(fdir, fname, data_format)


In [ ]:
# mean_subsample_img = np.squeeze(np.nanmean(data_n_meta['data'][::40,...], axis=0))
# my_bidi_corr_obj = bidi_offset_correction.bidi_offset_correction(mean_subsample_img)  # initialize data to object

# my_bidi_corr_obj.compute_mean_image()  # compute mean image across time
# bidi_offset = my_bidi_corr_obj.determine_bidi_offset()  # calculated bidirectional offset via fft cross-correlation

In [ ]:
# applies an offset to odd lines. Every time you run this, it will update the sima offsets file!!!
bidi_funcs.apply_bidi_corr_to_sima_offsets(fdir, fname, -manual_offset_value-1)

In [ ]:
# show mean image
dataset = sima.ImagingDataset.load(os.path.join(fdir, os.path.splitext(fname)[0] + '_mc.sima'))
sequence_data = dataset.sequences[0][:num_save_frames,...]

In [ ]:
# comment this out if you don't want to wait for the mean image to process and show in this notebook
plt.figure(figsize=(7,7))
plt.imshow(np.nanmean(np.squeeze(sequence_data), axis=0))

In [ ]:
# saves an h5 based on the motion correction shifts calculated by sima
data_to_save = np.empty([num_save_frames, dataset.frame_shape[1], dataset.frame_shape[2]])
frame_iter1 = iter(sequence_data)

# these next few lines just fill in the empty spaces
fill_gapscaller = bidi_funcs.fill_gaps(0, sequence_data, frame_iter1)
fill_gapscaller.send(None)

for frame_num in range(num_save_frames):
    data_to_save[frame_num, ...] = fill_gapscaller.send(frame_num).astype('uint8')

# save to an h5
sima_mc_bidi_outpath = os.path.join(fdir, fname + '_sima_mc.h5')
h5_write_bidi_corr = h5py.File(sima_mc_bidi_outpath, 'w')
h5_write_bidi_corr.create_dataset('imaging', data=data_to_save.astype('uint8'))
h5_write_bidi_corr.close()